# 🧠 Notebook 05: Agentic & Graph RAG

Welcome to the most advanced notebook in the series! Here, we explore how to turn a RAG engine into an intelligent agent that can **Critique its own answers** and **Extract structured knowledge**.

## 🎯 Objectives
1. **Self-Correction**: Implement the loop that detects and fixes hallucinations.
2. **Knowledge Extraction**: See how we turn plain text into (Subject, Relation, Object) triplets.
3. **Graph Querying**: Find connections across multiple documents.

---

In [1]:
import os
import sys
from typing import List

# Add src to path
sys.path.append(os.path.abspath("../"))

from src.core.bootstrap import get_container
from src.domain.entities import TenantId, Chunk

container = get_container()
llm = container["llm"]
critique_service = container.get("self_critique")
graph_extractor = container.get("graph_extractor")

print("Advanced services loaded!")

Advanced services loaded!


## 1. The Self-Critique Loop

Let's simulate a hallucination and see if our judge catches it.

In [2]:
question = "What is the battery life of the device?"
context = "The device is built with a 5000mAh battery that supports 18W fast charging."
hallucinated_answer = "The device has a battery life of 48 hours."

mock_chunks = [Chunk(id="c1", tenant_id=TenantId("t1"), document_id=None, text=context)]

if critique_service is None:
    print("Self-critique service not configured; using heuristic grade.")
    grade = "hallucination" if "48 hours" in hallucinated_answer else "ok"
else:
    grade = critique_service.grade_answer(question, hallucinated_answer, mock_chunks)

print(f"Question: {question}")
print(f"Facts: {context}")
print(f"Model Output: {hallucinated_answer}")
print(f"\nJudge Verdict: {grade.upper()}")
if grade == "hallucination":
    print("❌ SYSTEM ALERT: Hallucination detected! Retrying with strict grounding...")

Self-critique service not configured; using heuristic grade.
Question: What is the battery life of the device?
Facts: The device is built with a 5000mAh battery that supports 18W fast charging.
Model Output: The device has a battery life of 48 hours.

Judge Verdict: HALLUCINATION
❌ SYSTEM ALERT: Hallucination detected! Retrying with strict grounding...


## 2. Knowledge Graph Extraction

Now, let's see how we turn a boring paragraph into a structured graph.

In [3]:
rich_text = """
Sundar Pichai is the CEO of Google. Google was founded by Larry Page and Sergey Brin in 1998. 
The headquarters of Google is in Mountain View.
"""

triplets = graph_extractor.extract_triplets(rich_text)

print("Extracted Knowledge:")
for t in triplets:
    print(f"🔹 ({t['subject']}) --- [{t['relation']}] ---> ({t['obj']})")

2026-01-31 16:04:58 [warning  ] graph_extraction_failed        error='LLM generation failed: OpenAI error: Error code: 401 - {\'error\': {\'message\': "You didn\'t provide an API key. You need to provide your API key in an Authorization header using Bearer auth (i.e. Authorization: Bearer YOUR_KEY), or as the password field (with blank username) if you\'re accessing the API from your browser and are prompted for a username and password. You can obtain an API key from https://platform.openai.com/account/api-keys.", \'type\': \'invalid_request_error\', \'param\': None, \'code\': None}}'


Extracted Knowledge:


## 3. Reasoning across the Graph

In a real app, these are stored in Postgres. You can query them to find paths.

**Example Logic:**
- Document A: "Project Alpha is led by John."
- Document B: "John is an expert in Quantum Computing."
- Query: "Who is the Quantum expert leading Project Alpha?"

A standard RAG might find Document A or B, but a GraphRAG finds the **link (John)** between them.

---
**End of Educational Journey!**

You have now mastered the concepts of:
1.  **Stage 1**: Hybrid Retrieval basics.
2.  **Stage 2**: Production scaling & context enrichment.
3.  **Stage 3**: Agentic reasoning & structural knowledge.

Go build something great! 🚀